In [ ]:
using ProgressMeter, PyCall, PyPlot, Images, HDF5,NaNStatistics, Statistics, DSP, Lasso, JLD2
using _Data, _Math

In [ ]:
@pyimport numpy
@pyimport sklearn.decomposition as decomposition
@pyimport scipy.stats as stats
@pyimport matplotlib.colors as mpl_colors
@pyimport matplotlib.cm as cm 
@pyimport sklearn.cluster as cluster

In [ ]:
rc_params = PyDict(pyimport("matplotlib")["rcParams"]);
rc_params["font.sans-serif"] = ["Arial"];
rc_params["font.size"] = 7;
rc_params["lines.linewidth"] = 1;
rc_params["lines.markersize"] = 4;
rc_params["xtick.major.size"] = 2;
rc_params["ytick.major.size"] = 2;
rc_params["xtick.major.pad"] = 2;
rc_params["ytick.major.pad"] = 2;
# rc_params["axes.labelpad"] = 2;
rc_params["axes.spines.right"] = false
rc_params["axes.spines.top"] = false

cim(img::Matrix{UInt32}) = CairoImageSurface(img, Cairo.FORMAT_RGB24; flipxy = false) 
cim(img::Matrix{ARGB32}) = cim(reinterpret(UInt32, img))
cim(img::Matrix{RGB24}) = cim(reinterpret(UInt32, img))
cim(img::Matrix{UInt8}) = cim(Gray.(reinterpret(N0f8, img)))
cim(img::Array{UInt8,3}) = cim(RGB24.(reinterpret(N0f8, img[:,:,1]), reinterpret(N0f8, img[:,:,2]), reinterpret(N0f8, img[:,:,3])));downsample(img,n=4) = +([img[i:n:n*div(size(img,1)-1,n), j:n:n*div(size(img,2)-1,n)] for i = 1:n, j = 1:n]...)/(n*n);
downsample(img,n=4) = +([img[i:n:n*div(size(img,1)-1,n), j:n:n*div(size(img,2)-1,n)] for i = 1:n, j = 1:n]...)/(n*n);

In [ ]:
fig_path = "/home/chuyu/Notebooks/project_place_cell/figures/output/figure4/hedgehog_rotation_fishout"
mkpath(fig_path)

In [ ]:
include("../../../functions/func_map.jl")
include("../../../functions/func_stat.jl")
include("../../../functions/func_data.jl")
include("../../../functions/func_plot.jl")

In [ ]:
data_info_dict = load("/home/chuyu/Notebooks/project_place_cell/figures/chuyu/figure_data_info.jld2")


job_name = "hedgehog_rotation_bottom_out"
data_info_all = reduce(vcat, [data_info_dict[job_name]])

In [ ]:
analyzer = "chuyu"

In [ ]:
if job_name != "boundary_morphing"
    global compare_map_file_name = "compare_map_results_matchangle.h5"
else
    global compare_map_file_name = "compare_map_results_matchboundary.h5"
end


n_bins = 31

corr_trials_all_tog = []
corr_pv_all_normalized_tog = []
peak_shift_mm_tog = []
    
corr_trials_all_exp = []
corr_pv_all_normalized_exp = []
peak_shift_mm_exp = []
    
corr_trials_all_tog_control = []
corr_pv_all_normalized_tog_control = []
peak_shift_mm_tog_control = []
    
corr_trials_all_exp_control = []
corr_pv_all_normalized_exp_control = []
peak_shift_mm_exp_control = []


for which_data in 1:length(data_info_all)
    data_info = data_info_all[which_data]
    experiment_filename_1 = data_info[1]
    server_1 = data_info[2]
     experimenter = data_info[3]

    ds_save_1 = Dataset(experiment_filename_1, experimenter, gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
    ds_save_cy_1 = Dataset(experiment_filename_1, "chuyu", gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
    ds_save_analyzer_1 = Dataset(experiment_filename_1, analyzer, gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
    
    info_filename = joinpath(data_path(ds_save_cy_1), compare_map_file_name)
    file = h5open(info_filename, "r")
    tel_place_cell_index = HDF5.readmmap(file["tel_place_cell_index"])
    confined_place_cell_index = HDF5.readmmap(file["confined_place_cell_index"])
    tel_place_cell_index = HDF5.readmmap(file["tel_place_cell_index"])
    corr_trials_all = HDF5.readmmap(file["corr_trials_all"])
    pv_map_normalized = HDF5.readmmap(file["pv_map_dFF"])
    peak_shift_mm = HDF5.readmmap(file["peak_shift_mm"])
    close(file)
        
    info_filename = joinpath(data_path(ds_save_cy_1), "compare_map_results_early_late.h5")
    file = h5open(info_filename, "r")
    confined_place_cell_index_earlylate = HDF5.readmmap(file["confined_place_cell_index"])
    corr_trials_all_earlylate = HDF5.readmmap(file["corr_earlylate_all"])
    pv_map_normalized_earlylate = HDF5.readmmap(file["pv_map_dFF"])
    peak_shift_mm_earlylate = HDF5.readmmap(file["peak_shift_early_late_mm"])
    close(file)
        

    corr_pv_all_normalized = pv_map_normalized[isfinite.(pv_map_normalized_earlylate).*isfinite.(pv_map_normalized)]
    corr_pv_all_normalized_earlylate = pv_map_normalized_earlylate[isfinite.(pv_map_normalized_earlylate).*isfinite.(pv_map_normalized)]

    density_corr_trials, bins = numpy.histogram(corr_trials_all[tel_place_cell_index], bins= numpy.linspace(-1,1,n_bins), density=true)
    density_corr_pv_normalized, bins = numpy.histogram(corr_pv_all_normalized, bins= numpy.linspace(-1,1,n_bins), density=true)
    density_shift, bins = numpy.histogram(peak_shift_mm[whether_in(confined_place_cell_index, confined_place_cell_index_earlylate)], bins= numpy.linspace(0, 50,n_bins), density=true)
 
    append!(corr_trials_all_tog, [density_corr_trials])
    append!(corr_pv_all_normalized_tog, [density_corr_pv_normalized])
    append!(peak_shift_mm_tog, [density_shift])
        
        
    append!(corr_trials_all_exp, [corr_trials_all[tel_place_cell_index]])
    append!(corr_pv_all_normalized_exp, [pv_map_normalized])
    append!(peak_shift_mm_exp, [peak_shift_mm[whether_in(confined_place_cell_index, confined_place_cell_index_earlylate)]])
 

        
    density_corr_trials, bins = numpy.histogram(corr_trials_all_earlylate[tel_place_cell_index], bins= numpy.linspace(-1,1,n_bins), density=true)
    density_corr_pv_normalized, bins = numpy.histogram(corr_pv_all_normalized_earlylate, bins= numpy.linspace(-1,1,n_bins), density=true)
    density_shift, bins = numpy.histogram(peak_shift_mm_earlylate[whether_in(confined_place_cell_index_earlylate, confined_place_cell_index)], bins= numpy.linspace(0, 50,n_bins), density=true)

    append!(corr_trials_all_tog_control, [density_corr_trials])
    append!(corr_pv_all_normalized_tog_control, [density_corr_pv_normalized])
    append!(peak_shift_mm_tog_control, [density_shift])
        
        
    append!(corr_trials_all_exp_control, [corr_trials_all_earlylate[tel_place_cell_index]])
    append!(corr_pv_all_normalized_exp_control, [pv_map_normalized_earlylate])
    append!(peak_shift_mm_exp_control, [peak_shift_mm_earlylate[whether_in(confined_place_cell_index_earlylate, confined_place_cell_index)]])
 
    
    
end




In [ ]:
using Printf
function plot_mean_density(bins, y_tog; plot_color="k", shade_color="gray", line_color="k", plot_median = true)
    
    n_bins = length(bins)
    x = (bins[1:end-1] + bins[2:end])/2
    y = mean(y_tog)
    ymax = y + std(y_tog)
    ymin = y - std(y_tog);
    
    cumsum_y_mean = cumsum(mean(y_tog)*(maximum(bins) - minimum(bins))/(n_bins-1))

    x_median = numpy.round(numpy.interp(0.5, cumsum_y_mean, x), 2)
    x_14 = numpy.round(numpy.interp(0.25, cumsum_y_mean, x), 2)
    x_34 = numpy.round(numpy.interp(0.75, cumsum_y_mean, x), 2)
    
    
    plot(x, y, color= plot_color)
    fill_between(x, ymax, ymin, color=shade_color, alpha=0.3)
    if plot_median
        axvline(x_median, color=line_color, linestyle="dashed")
        # axvline(x_median, color=line_color, label= "$x_median ($x_14 - $x_34)")
    end
    
    

end

In [ ]:
fig_path

In [ ]:


fig = figure(figsize=(0.8, 0.8))
bins = numpy.linspace(-1,1,n_bins)
plot_mean_density(bins, corr_trials_all_tog_control)
plot_mean_density(bins, corr_trials_all_tog;plot_color="r", shade_color="r", line_color="r")
xlabel("PF cor.", labelpad = 0)
ylabel("Density", labelpad = -4)
xlim(-1,1)
legend(frameon=false, loc= (0,1))
yticks([0,3])

fig.savefig(joinpath(fig_path, "PF_correlation.pdf"), bbox_inches="tight",transparent = true);



fig = figure(figsize=(0.8, 0.8))
bins = numpy.linspace(-1,1,n_bins)
x = (bins[1:end-1] + bins[2:end])/2
plot_mean_density(bins, corr_pv_all_normalized_tog_control)
plot_mean_density(bins, corr_pv_all_normalized_tog;plot_color="r", shade_color="r", line_color="r")
xlabel("PV cor.", labelpad = 0)
xlim(-1,1)
legend(frameon=false, loc= (0,1))
fig.savefig(joinpath(fig_path, "PV_correlation.pdf"), bbox_inches="tight",transparent = true);

fig = figure(figsize=(0.8, 0.8))
bins = numpy.linspace(0, 50,n_bins)
x = (bins[1:end-1] + bins[2:end])/2
plot_mean_density(bins, peak_shift_mm_tog_control)
plot_mean_density(bins, peak_shift_mm_tog;plot_color="r", shade_color="r", line_color="r")
xlabel("PF shift (mm)", labelpad = 0)
xlim(0, 50)
legend(frameon=false, loc= (0,1))
fig.savefig(joinpath(fig_path, "peak_shift.pdf"), bbox_inches="tight",transparent = true);


In [ ]:
@pyimport pandas as pd
function generate_dict(bins, y_tog, name, data_dict)
    y_mean = mean(y_tog)
    y_std = std(y_tog)
    
    data_dict["$(name) bins"] = bins
    data_dict["$(name) mean"] = y_mean
    data_dict["$(name) std"] = y_std


    
    return data_dict
end

function fill_length(vec, desired_length)
    [vec; fill(NaN, desired_length-length(vec))]
end

In [ ]:
data_dict = Dict()
generate_dict(bins, corr_trials_all_tog_control, "PF correlation S1 early vs late", data_dict)
generate_dict(bins, corr_trials_all_tog, "PF correlation S1 vs S2", data_dict)
generate_dict(bins, corr_pv_all_normalized_tog_control, "PV correlation S1 early vs late", data_dict)
generate_dict(bins, corr_pv_all_normalized_tog, "PV correlation S1 vs S2", data_dict)
generate_dict(bins, peak_shift_mm_tog_control, "PF shift (mm) S1 early vs late", data_dict)
generate_dict(bins, peak_shift_mm_tog, "PF shift (mm) S1 vs S2", data_dict)

max_length = maximum([length(data_dict[key]) for key in keys(data_dict)])

for key in keys(data_dict)
    data_dict[key] = fill_length(data_dict[key], max_length)
end

df = pd.DataFrame.from_dict(data_dict)
df = df.sort_index(axis=1)

In [ ]:
@pyimport os
@pyimport openpyxl

save_path = "../../../figures/source_data/Source Data Fig. 5.xlsx"
sheet_name = "Fig. 5d"

exists = os.path.isfile(save_path)
if exists
    writer = pd.ExcelWriter(save_path, engine = "openpyxl", mode="a")
    df.to_excel(writer, sheet_name = sheet_name)
    writer.close()
else
    df.to_excel(save_path, sheet_name)  
end

In [ ]:
n_bins = 31

corr_trials_all_tog = []
corr_pv_all_normalized_tog = []
peak_shift_mm_tog = []
specificity_diff_tog_1 = []
specificity_diff_tog_2 = []
for which_data in 1:length(data_info_all)
    data_info = data_info_all[which_data]
    experiment_filename_1 = data_info[1]
    server_1 = data_info[2]
     experimenter = data_info[3]

    ds_save_1 = Dataset(experiment_filename_1, experimenter, gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
    ds_save_cy_1 = Dataset(experiment_filename_1, "chuyu", gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
    ds_save_analyzer_1 = Dataset(experiment_filename_1, analyzer, gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
    
    
    info_filename = joinpath(data_path(ds_save_cy_1), "compare_map_results_matchangle.h5")
    file = h5open(info_filename, "r")
    tel_place_cell_index_1 = HDF5.readmmap(file["tel_place_cell_index_1"])
    tel_place_cell_index_2 = HDF5.readmmap(file["tel_place_cell_index_2"])
    tel_place_cell_index = HDF5.readmmap(file["tel_place_cell_index"])
    confined_place_cell_index = HDF5.readmmap(file["confined_place_cell_index"])
    corr_trials_all = HDF5.readmmap(file["corr_trials_all"])
    pv_map_normalized = HDF5.readmmap(file["pv_map_dFF"])
    corr_pv_all_normalized = pv_map_normalized[isfinite.(pv_map_normalized)]
    peak_shift_mm = HDF5.readmmap(file["peak_shift_mm"])
    specificity_1 = HDF5.readmmap(file["specificity_1"])
    specificity_2 = HDF5.readmmap(file["specificity_2"])
    close(file)
    
    
    info_filename = joinpath(data_path(ds_save_cy_1), "compare_map_results_early_late.h5")
    file = h5open(info_filename, "r")
    confined_place_cell_index_early_late = HDF5.readmmap(file["confined_place_cell_index"])
    close(file)

    
    

    
    append!(corr_trials_all_tog, [corr_trials_all[tel_place_cell_index]])
    append!(corr_pv_all_normalized_tog, [pv_map_normalized])
    append!(peak_shift_mm_tog, [peak_shift_mm[whether_in(confined_place_cell_index, confined_place_cell_index_early_late)]])
 
    
end




In [ ]:
n_bins = 31

corr_trials_all_tog_control = []
corr_pv_all_normalized_tog_control = []
peak_shift_mm_tog_control = []

for which_data in 1:length(data_info_all)
    data_info = data_info_all[which_data]
    experiment_filename_1 = data_info[1]
    server_1 = data_info[2]
     experimenter = data_info[3]

    ds_save_1 = Dataset(experiment_filename_1, experimenter, gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
    ds_save_cy_1 = Dataset(experiment_filename_1, "chuyu", gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
    ds_save_analyzer_1 = Dataset(experiment_filename_1, analyzer, gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
    
    info_filename = joinpath(data_path(ds_save_cy_1), "compare_map_results_matchangle.h5")
    file = h5open(info_filename, "r")
    tel_place_cell_index = HDF5.readmmap(file["tel_place_cell_index"])
    tel_place_cell_index_1 = HDF5.readmmap(file["tel_place_cell_index_1"])
    tel_place_cell_index_2 = HDF5.readmmap(file["tel_place_cell_index_2"])
    confined_place_cell_index = HDF5.readmmap(file["confined_place_cell_index"])
    close(file)
    
    
    info_filename = joinpath(data_path(ds_save_cy_1), "compare_map_results_early_late.h5")
    file = h5open(info_filename, "r")
    confined_place_cell_index_early_late = HDF5.readmmap(file["confined_place_cell_index"])
    corr_trials_all = HDF5.readmmap(file["corr_earlylate_all"])
    pv_map_normalized = HDF5.readmmap(file["pv_map_dFF"])
    corr_pv_all_normalized = pv_map_normalized[isfinite.(pv_map_normalized)]
    peak_shift_mm = HDF5.readmmap(file["peak_shift_early_late_mm"])
    specificity_1 = HDF5.readmmap(file["specificity_1_early"])
    specificity_2 = HDF5.readmmap(file["specificity_1_late"])
    close(file)

    
    append!(corr_trials_all_tog_control, [corr_trials_all[tel_place_cell_index]])
    append!(corr_pv_all_normalized_tog_control, [pv_map_normalized])
    append!(peak_shift_mm_tog_control, [peak_shift_mm[whether_in(confined_place_cell_index_early_late, confined_place_cell_index)]])
end


In [ ]:
for which_fish = 1:length(corr_trials_all_tog)
x = reduce(vcat, corr_trials_all_tog[which_fish])
y = reduce(vcat, corr_trials_all_tog_control[which_fish])

valid_neuron = isfinite.(x) .* isfinite.(y)
_, p_value = stats.wilcoxon(x[valid_neuron], y[valid_neuron], alternative = "less")
println(p_value)
end

In [ ]:
x = reduce(vcat, corr_trials_all_tog)
y = reduce(vcat, corr_trials_all_tog_control)

valid_neuron = isfinite.(x) .* isfinite.(y)
stats.wilcoxon(x[valid_neuron], y[valid_neuron], alternative = "less")

In [ ]:
for which_fish = 1:length(corr_pv_all_normalized_tog_control)
x = reduce(vcat, corr_pv_all_normalized_tog[which_fish])
y = reduce(vcat, corr_pv_all_normalized_tog_control[which_fish])

valid_neuron = isfinite.(x) .* isfinite.(y)
_, p_value = stats.wilcoxon(x[valid_neuron], y[valid_neuron], alternative = "less")
println(p_value)
end

In [ ]:
x = reduce(vcat, corr_pv_all_normalized_tog)
y = reduce(vcat, corr_pv_all_normalized_tog_control)

valid_neuron = isfinite.(x) .* isfinite.(y)
stats.wilcoxon(x[valid_neuron], y[valid_neuron], alternative = "less")

In [ ]:
for which_fish = 1:length(peak_shift_mm_tog_control)
x = reduce(vcat, peak_shift_mm_tog[which_fish])
y = reduce(vcat, peak_shift_mm_tog_control[which_fish])

valid_neuron = isfinite.(x) .* isfinite.(y)
_, p_value = stats.wilcoxon(x[valid_neuron], y[valid_neuron], alternative = "greater")
println(p_value)
end

In [ ]:
x = reduce(vcat, peak_shift_mm_tog)
y = reduce(vcat, peak_shift_mm_tog_control)

valid_neuron = isfinite.(x) .* isfinite.(y)
stats.wilcoxon(x[valid_neuron], y[valid_neuron], alternative = "greater")